# VarianceComponentModels.jl

In [1]:
using VarianceComponentModels

VarianceComponentModels.jl implements computation routines for fitting and testing variance component model of form
$$
    \text{vec}(Y) \sim {\cal N}(X B, \Sigma_1 \otimes V_1 + \cdots + \Sigma_m \otimes V_m).
$$
In this model, **data** is  
* `Y`: `n x d` response matrix  
* `X`: `n x p` covariate matrix  
* `V1,...,Vm`: `m` `n x n` covariance matrices  

**Parameters** are  
* `B`: `p x d` mean parameter matrix  
* `Σ1,...,Σm`: `m` `d x d` variance components

## Maximum likelihood estimation (MLE)

For demonstration, we generate a random data set.

In [2]:
# generate data from a d-variate response variane component model
srand(123)
n = 1000   # no. observations
d = 2     # dimension of responses
m = 2     # no. variance components
p = 2     # no. covariates
X = randn(n, p) # design matrix
B = ones(p, d)  # mean component regresison coefficient
Σ = ntuple(x -> zeros(d, d), m) # A tuple of m variance component parameters
for i in 1:m
  Σi = randn(d, d)
  copy!(Σ[i], Σi' * Σi)
end
V = ntuple(x -> zeros(n, n), m) # A tuple of m covariance matrices
for i = 1:m-1
  Vi = randn(n, 50)
  copy!(V[i], Vi * Vi')
end
copy!(V[m], eye(n)) # last covarianec matrix is idendity
# form Ω
Ω = zeros(n*d, n*d)
for i = 1:m
  Ω += kron(Σ[i], V[i])
end
Ωchol = cholfact(Ω)
Y = X * B + reshape(Ωchol[:L] * randn(n*d), n, d) # responses

1000x2 Array{Float64,2}:
  4.44498     2.49669 
  0.239013    5.78242 
 -7.89026    11.3229  
 -1.14952    -1.99452 
  2.07618    -4.97312 
  0.72998    -0.710448
  4.31562    -1.13738 
 -9.52372     8.97905 
 -3.48908     6.85469 
 14.5379    -29.2101  
  4.21264    -6.69222 
  2.17465    -7.16013 
 -0.484665   -3.31723 
  ⋮                   
 -6.50965    -0.272068
  9.63446   -11.2917  
  3.9017    -10.4169  
 -1.87012     4.04243 
  5.33523    -9.45383 
  0.633076    3.54368 
 -2.41406     3.75242 
 -4.81822     5.10283 
  3.93386     0.107781
  4.4098     -9.14736 
 -5.17633     0.593472
 -4.90908    10.5114  

To find the MLE of parameters $(B,\Sigma_1,\ldots,\Sigma_m)$, we follow 3 steps:  
**Step 1**. Construct an instance of `VarianceComponentVariate`, which consists of responses $Y$, covariate matrix $X$, and a tuple of covariance matrices $V$. The last covariance matrix must be positive definite and usually is the identity matrix. In absence of covariates $X$, we can simply initialize by `vcdata = VarianceComponentVariate(Y, V)`.

In [3]:
vcdata = VarianceComponentVariate(Y, X, V)
fieldnames(vcdata)

3-element Array{Symbol,1}:
 :Y
 :X
 :V

In [4]:
vcdata

VarianceComponentModels.VarianceComponentVariate{Float64,2,Array{Float64,2},Array{Float64,2},Array{Float64,2}}(1000x2 Array{Float64,2}:
  4.44498     2.49669 
  0.239013    5.78242 
 -7.89026    11.3229  
 -1.14952    -1.99452 
  2.07618    -4.97312 
  0.72998    -0.710448
  4.31562    -1.13738 
 -9.52372     8.97905 
 -3.48908     6.85469 
 14.5379    -29.2101  
  4.21264    -6.69222 
  2.17465    -7.16013 
 -0.484665   -3.31723 
  ⋮                   
 -6.50965    -0.272068
  9.63446   -11.2917  
  3.9017    -10.4169  
 -1.87012     4.04243 
  5.33523    -9.45383 
  0.633076    3.54368 
 -2.41406     3.75242 
 -4.81822     5.10283 
  3.93386     0.107781
  4.4098     -9.14736 
 -5.17633     0.593472
 -4.90908    10.5114  ,1000x2 Array{Float64,2}:
  1.19027      1.14112    
  2.04818      0.597106   
  1.14265     -1.30405    
  0.459416    -1.2566     
 -0.396679    -0.706518   
 -0.664713     1.38036    
  0.980968    -0.826625   
 -0.0754831    0.371915   
  0.273815     0.292571  

**Step 2**. Construct an instance of `VarianceComponentModel`, which initializes the mean parameters $B$ to be zero and the tuple of variance component parameters $\Sigma$ to be `(eye(d),...,eye(d))`. 

In [5]:
vcmodel = VarianceComponentModel(vcdata)
fieldnames(vcmodel)

2-element Array{Symbol,1}:
 :B
 :Σ

In [6]:
vcmodel

VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}(2x2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0,(
2x2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0,

2x2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0))

When better initial guess is available, we can initialize by calling `vcmodel = VarianceComponentModel(B0, Σ0)` directly.

**Step 3**. Call optmization routine `fit_mle!`.

In [7]:
vcmodel_mle = deepcopy(vcmodel)
@time logl, vcmodel_mle, Σse, Σcov, Bse, Bcov = fit_mle!(vcmodel_mle, vcdata; algo = :MM);


     MM Algorithm
  Iter      Objective  
--------  -------------
       0  -7.348297e+03
       1  -4.102367e+03
       2  -3.745567e+03
       3  -3.652392e+03
       4  -3.627744e+03
       5  -3.621170e+03
       6  -3.619381e+03
       7  -3.618878e+03
       8  -3.618730e+03
       9  -3.618684e+03
      10  -3.618670e+03

  7.866850 seconds (12.99 M allocations: 622.175 MB, 1.72% gc time)


The output of `fit_mle!` contains:  
* final log-likelihood  

In [8]:
logl

-3618.661776037447

* fitted model

In [9]:
vcmodel_mle

VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}(2x2 Array{Float64,2}:
 1.14929   1.00139
 0.924768  1.02485,(
2x2 Array{Float64,2}:
  0.302279  -0.478398
 -0.478398   0.803237,

2x2 Array{Float64,2}:
  5.86133   -0.586939
 -0.586939   0.586382))

* standard errors of the estimated varianec component parameters

In [10]:
Σse

(
2x2 Array{Float64,2}:
 0.06167    0.0995727
 0.0995727  0.16077  ,

2x2 Array{Float64,2}:
 0.268915  0.085063
 0.085063  0.026905)

* covariance matrix of the variance component parameters estimates

In [11]:
Σcov

8x8 Array{Float64,2}:
  0.00380319  -0.00590812  -0.00590812  …   7.45979e-6   -7.52483e-7 
 -0.00590812   0.00991472   0.00917805     -7.52483e-7    7.55745e-7 
 -0.00590812   0.00917805   0.00991472     -7.49213e-6    7.55745e-7 
  0.00917805  -0.0154021   -0.0154021       7.55745e-7   -7.59023e-7 
 -7.39532e-5   7.45979e-6   7.45979e-6     -0.00724213    0.000725236
  7.45979e-6  -7.49213e-6  -7.52483e-7  …   0.000725236  -0.000724568
  7.45979e-6  -7.52483e-7  -7.49213e-6      0.00723572   -0.000724568
 -7.52483e-7   7.55745e-7   7.55745e-7     -0.000724568   0.000723881

* standard errors of the estimated mean parameters

In [12]:
Bse

2x2 Array{Float64,2}:
 0.0760538  0.0251806
 0.0769259  0.0252831

* covariance matrix of the mean parameter estimates

In [13]:
Bcov

4x4 Array{Float64,2}:
  0.00578418  -6.1406e-5    -0.00061133    3.61347e-6 
 -6.1406e-5    0.00591759    3.61347e-6   -0.000619951
 -0.00061133   3.61347e-6    0.000634064  -1.76949e-6 
  3.61347e-6  -0.000619951  -1.76949e-6    0.000639237

## Restricted maximum likelihood estimation (REML)

[REML (restricted maximum likelihood estimation)](https://en.wikipedia.org/wiki/Restricted_maximum_likelihood) is a popular alternative to the MLE. To find the REML of a variane component model, we replace the above step 3 by  

**Step 3**. Call optmization routine `fit_reml!`.

In [14]:
vcmodel_reml = deepcopy(vcmodel)
@time logl, vcmodel_reml, Σse, Σcov, Bse, Bcov = fit_reml!(vcmodel_reml, vcdata; algo = :MM);


     MM Algorithm
  Iter      Objective  
--------  -------------
       0  -5.297304e+03
       1  -3.954293e+03
       2  -3.715870e+03
       3  -3.663101e+03
       4  -3.650121e+03
       5  -3.646663e+03
       6  -3.645672e+03
       7  -3.645367e+03
       8  -3.645268e+03
       9  -3.645233e+03
      10  -3.645221e+03

  0.499879 seconds (767.13 k allocations: 44.538 MB, 1.83% gc time)


The output of `fit_reml!` contains:

* final log-likelihood at REML estimate

In [15]:
logl

-3622.050155483128

* REML estimates

In [16]:
vcmodel_reml

VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}(2x2 Array{Float64,2}:
 1.08201  1.05525
 0.90694  1.00679,(
2x2 Array{Float64,2}:
  0.301641  -0.477617
 -0.477617   0.802105,

2x2 Array{Float64,2}:
  5.88057   -0.610115
 -0.610115   0.61958 ))

* standard errors of the estimated varianec component parameters

In [17]:
Σse

(
2x2 Array{Float64,2}:
 0.0615463  0.0994049
 0.0994049  0.160551 ,

2x2 Array{Float64,2}:
 0.269798   0.0875812
 0.0875812  0.0284283)

* covariance matrix of the variance component parameters estimates

In [18]:
Σcov

8x8 Array{Float64,2}:
  0.00378795  -0.00588696  -0.00588696  …   7.78026e-6   -8.13244e-7 
 -0.00588696   0.00988133   0.00914906     -8.13244e-7    8.30149e-7 
 -0.00588696   0.00914906   0.00988133     -7.94198e-6    8.30149e-7 
  0.00914906  -0.0153568   -0.0153568       8.30149e-7   -8.47406e-7 
 -7.44333e-5   7.78026e-6   7.78026e-6     -0.00755281    0.000783641
  7.78026e-6  -7.94198e-6  -8.13244e-7  …   0.000783641  -0.00079582 
  7.78026e-6  -8.13244e-7  -7.94198e-6      0.00767047   -0.00079582 
 -8.13244e-7   8.30149e-7   8.30149e-7     -0.00079582    0.000808168

* standard errors of the estimated mean parameters

In [19]:
Bse

2x2 Array{Float64,2}:
 0.0761855  0.0258832
 0.0770579  0.0259886

* covariance matrix of the mean parameter estimates

In [20]:
Bcov

4x4 Array{Float64,2}:
  0.00580423   -6.15234e-5   -0.000636032   3.71134e-6 
 -6.15234e-5    0.00593793    3.71134e-6   -0.000644911
 -0.000636032   3.71134e-6    0.000669939  -1.87083e-6 
  3.71134e-6   -0.000644911  -1.87083e-6    0.00067541 

## Optimization algorithms

Finding the MLE or REML of variance component models is a non-trivial nonlinear optimization problem, due to the non-convexity of objective function and the positive semi-definiteness constraint of variane component parameters $\Sigma_1,\ldots,\Sigma_m$. Here are some details of the implementation. 

In general the optimization algorithm needs to invert the $nd$ by $nd$ overall covariance matrix $\Omega = \Sigma_1 \otimes V_1 + \cdots + \Sigma_m \otimes V_m$ in each iteration. Inverting a matrix is an expensive operation with $O(n^3 d^3)$ floating operations. When there are only **two** varianec components ($m=2$), this tedious task can be avoided by taking one (generalized) eigendecomposion of $(V_1, V_2)$ and rotating data $(Y, X)$ by the eigen-vectors. 

In [21]:
vcdatarot = TwoVarCompVariateRotate(vcdata)
fieldnames(vcdatarot)

4-element Array{Symbol,1}:
 :Yrot    
 :Xrot    
 :eigval  
 :logdetV2

Two optimization algorithms are implemented: Fisher scoring (function `mle_fs!`) and the [minorization-maximization (MM) algorithm](http://arxiv.org/abs/1509.07426) (function `mle_mm!`). Both take the rotated data as input. These two functions give finer control of the optimization algorithms. Generally speaking, MM algorithm is more stable while Fisher scoring (if converges) yields more accurate answer.

In [22]:
vcmodel_mm = deepcopy(vcmodel)
@time mle_mm!(vcmodel_mm, vcdatarot; maxiter=10000, funtol=1e-8, verbose = true)
vcmodel_mm


     MM Algorithm
  Iter      Objective  
--------  -------------
       0  -7.348297e+03
       1  -4.102367e+03
       2  -3.745567e+03
       3  -3.652392e+03
       4  -3.627744e+03
       5  -3.621170e+03
       6  -3.619381e+03
       7  -3.618878e+03
       8  -3.618730e+03
       9  -3.618684e+03
      10  -3.618670e+03

  0.164586 seconds (773.53 k allocations: 28.231 MB, 2.89% gc time)


VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}(2x2 Array{Float64,2}:
 1.14929   1.00139
 0.924768  1.02485,(
2x2 Array{Float64,2}:
  0.302279  -0.478398
 -0.478398   0.803237,

2x2 Array{Float64,2}:
  5.86133   -0.586939
 -0.586939   0.586382))

Fisher scoring (function `mle_fs!`) uses either [Ipopt.jl](https://github.com/JuliaOpt/Ipopt.jl) (keyword `solver=:Ipopt`) or [KNITRO.jl](https://github.com/JuliaOpt/KNITRO.jl) (keyword `solver=:Knitro`) as the backend solver. In practice we find better performance of Knitro over Ipopt. However Knitro is a commercial software and users need to follow instructions at [KNITRO.jl](https://github.com/JuliaOpt/KNITRO.jl) for proper functioning. Ipopt is open source and installation of [Ipopt.jl](https://github.com/JuliaOpt/Ipopt.jl) package alone is sufficient.

In [23]:
vcmodel_fs = deepcopy(vcmodel)
@time mle_fs!(vcmodel_fs, vcdatarot; solver=:Knitro, maxiter=1000, verbose=true)
vcmodel_fs


Knitro 10.1.0 STUDENT LICENSE (problem size limit = 300)

            Student License
       (NOT FOR COMMERCIAL USE)
         Artelys Knitro 10.1.0

Knitro presolve eliminated 0 variables and 0 constraints.

algorithm:            1
The problem is identified as bound constrained only.
Knitro changing bar_initpt from AUTO to 3.
Knitro changing bar_murule from AUTO to 4.
Knitro changing bar_penaltycons from AUTO to 1.
Knitro changing bar_penaltyrule from AUTO to 2.
Knitro changing bar_switchrule from AUTO to 1.
Knitro changing linsolver from AUTO to 2.

Problem Characteristics                    ( Presolved)
-----------------------
Objective goal:  Maximize
Number of variables:                    10 (        10)
    bounded below:                       4 (         4)
    bounded above:                       0 (         0)
    bounded below and above:             0 (         0)
    fixed:                               0 (         0)
    free:                                6 (         6)

### Could not find a valid license.
    Your machine ID is 1f-aa-f6-5b-46.
    Please contact licensing@artelys.com or your local distributor to obtain a license.
    If you already have a license, please execute `get_machine_ID -v` and send the output to support.


VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}(2x2 Array{Float64,2}:
 1.14928   1.00139
 0.924761  1.02485,(
2x2 Array{Float64,2}:
  0.30227   -0.478407
 -0.478407   0.803239,

2x2 Array{Float64,2}:
  5.86164   -0.586964
 -0.586964   0.586375))

In [24]:
vcmodel_ipopt = deepcopy(vcmodel)
@time mle_fs!(vcmodel_ipopt, vcdatarot; solver=:Ipopt, maxiter=1000, verbose=true)
vcmodel_ipopt


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       31

Total number of variables............................:       10
                     variables with only lower bounds:        4
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}(2x2 Array{Float64,2}:
 1.15708   1.00104
 0.930692  1.02464,(
2x2 Array{Float64,2}:
  0.545248  -0.875337
 -0.875337   1.57689 ,

2x2 Array{Float64,2}:
  5.90114   -0.592005
 -0.592005   0.590786))

## Starting point

A good starting point helps with successful optimization. Here are a few strategies.

* For $d>1$ (multivariate response), initialize $B, \Sigma$ from univariate estimates.  
* Use REML estimate as starting point for MLE.  